# Non-Equilibrium Green's functions and Transport

## Quantum Matter - Materials & Concepts Summer School 2022

### Hands-on session

In this session, we will

1. Evaluate the self-energy due to a semi-infinite 1D tight-binding chain 
2. Evaluate transport through a single level system
3. Explore the effects of electron-electron interactions at a mean-field level

This notebook uses the [Julia](https://julialang.org) programming language.
It is convenient to load the package `LinearAlgebra`, to solve linear algebra problems. 
For plotting we will use the package `PyPlot`.

In [2]:
using Pkg # this will load Julia's package manager
Pkg.add("PyPlot") # this will download and install the package PyPlot
Pkg.add("QuadGK") # pacage to do integrations
#(it only needs to be run if the package is not installed)

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [3]:
using LinearAlgebra, PyPlot, QuadGK

## 1. Evaluation of self-energy of a semi-infinite tight-binding chain

### 1.1. Bandstruture evaluation of a 1D tight-binding chain

We will consider a infinite chain, described by the Hamiltonian

$$
H_{\text{infinito}}=\left[\begin{array}{cccccc}
 & \ddots\\
\ddots & \ddots & \boldsymbol{v}\\
 & \boldsymbol{v}^{\dagger} & \boldsymbol{h} & \boldsymbol{v}\\
 &  & \boldsymbol{v}^{\dagger} & \boldsymbol{h} & \boldsymbol{v}\\
 &  &  & \boldsymbol{v}^{\dagger} & \ddots & \ddots\\
 &  &  &  & \ddots
\end{array}\right].
$$

To evaluate the bandstructure, we must solve the eigenvalue problem:

$$
\left(\boldsymbol{h}+e^{ik}\boldsymbol{v}+e^{-ik}\boldsymbol{v}^{\dagger}\right)\Psi_{k}=E_{k}\Psi_{k}.
$$

### Problem 1.1.a
Write a function that given $\boldsymbol{h}$ and $\boldsymbol{v}$ evaluates the bandstructure for a infinite 1D chain.

For convinience we can store the Hamiltonian of the chain with the structure:

## 1.2 Recursive evaluation of the self-energy

Now let us consider a semi-infinite chain, with Hamiltonian:

$$
H_{\text{semi}}=\left[\begin{array}{cccc}
\boldsymbol{h} & \boldsymbol{v}\\
\boldsymbol{v}^{\dagger} & \boldsymbol{h} & \boldsymbol{v}\\
 & \boldsymbol{v}^\dagger & \ddots & \ddots\\
 & \ddots
\end{array}\right]
$$

We want to evaluate the self-energy: $\Sigma^R = \boldsymbol{v}\boldsymbol{g}_{1,1}^R \boldsymbol{v}^\dagger$,

where the Green's function is solution of:

$$
\left[\begin{array}{cccc}
E-\boldsymbol{h} & -\boldsymbol{v}\\
-\boldsymbol{v}^{\dagger} & E-\boldsymbol{h} & -\boldsymbol{v}\\
 & -\boldsymbol{v}^{\dagger} & \ddots & \ddots\\
 &  & \ddots
\end{array}\right]\left[\begin{array}{cccc}
\boldsymbol{g}_{1,1}^{R} & \boldsymbol{g}_{1,2}^{R} & \boldsymbol{g}_{1,3}^{R} & \cdots\\
\boldsymbol{g}_{2,1}^{R} & \boldsymbol{g}_{2,2}^{R} & \boldsymbol{g}_{2,3}^{R} & \cdots\\
\boldsymbol{g}_{3,1}^{R} & \boldsymbol{g}_{3,2}^{R} & \boldsymbol{g}_{3,3}^{R} & \cdots\\
\vdots & \vdots & \vdots & \ddots
\end{array}\right]=\left[\begin{array}{cccc}
\boldsymbol{1}\\
 & \boldsymbol{1}\\
 &  & \boldsymbol{1} & \ddots\\
 &  & \ddots & \ddots
\end{array}\right]
$$

The recursive method is based on the fact that a semi-infinite chain can be seen as one site coupled to another equal semi-infinite chain. Treating the hopping between the first site and the rest of the chain as a perturbation, we obtain the Lippmann-Schwinger equations: 

$$
\begin{align*}
\boldsymbol{g}^{R} & =\tilde{\boldsymbol{g}}^{R}+\tilde{\boldsymbol{g}}^{R}\cdot\boldsymbol{V}\cdot\boldsymbol{g}^{R}\\
\boldsymbol{g}^{R} & =\tilde{\boldsymbol{g}}^{R}+\boldsymbol{g}^{R}\cdot\boldsymbol{V}\cdot\tilde{\boldsymbol{g}}^{R}
\end{align*}
$$

where:
$$
\tilde{\boldsymbol{g}}^{R}=\left[\begin{array}{cccc}
E-\boldsymbol{h} & \boldsymbol{0}\\
\boldsymbol{0} & E-\boldsymbol{h} & -\boldsymbol{v}\\
 & -\boldsymbol{v}^{\dagger} & \ddots & \ddots\\
 &  & \ddots
\end{array}\right]^{-1}
$$

is the Green's functions of the system where the first site is desconnected from the rest of the chain and
$$
\boldsymbol{V}=\left[\begin{array}{cccc}
\boldsymbol{0} & \boldsymbol{v}\\
\boldsymbol{v}^{\dagger} & \boldsymbol{0} & \boldsymbol{0}\\
 & \boldsymbol{0} & \boldsymbol{0} & \ddots\\
 &  & \ddots & \ddots
\end{array}\right]
$$

connects the first site to the rest of the chain.

From these we obtain the following relations:
$$
\begin{align*}
\boldsymbol{g}_{1,1}^{R} & =\tilde{\boldsymbol{g}}_{1,1}^{R}+\tilde{\boldsymbol{g}}_{1,1}^{R}\cdot\boldsymbol{V}_{1,2}\cdot\boldsymbol{g}_{2,1}^{R}\\
\boldsymbol{g}_{2,1}^{R} & =\tilde{\boldsymbol{g}}_{2,2}^{R}\cdot\boldsymbol{V}_{2,1}\cdot\boldsymbol{g}_{1,1}^{R}
\end{align*}
$$
Which can be combined yielding:
$$
\boldsymbol{g}_{1,1}^{R}=\left[E-\boldsymbol{h}-\boldsymbol{v}\cdot\tilde{\boldsymbol{g}}_{2,2}^{R}\cdot\boldsymbol{v}^{\dagger}\right]^{-1}
$$

It appears we have not made any progress, since we do not know $\tilde{\boldsymbol{g}}_{2,2}^{R}$. However, we notice that $\tilde{\boldsymbol{g}}_{2,2}^{R}$ is the on-site Green's function for the first site of a semi-infinite chain, which is preciselly what we wish to evaluate. Therefore, we have have that $\tilde{\boldsymbol{g}}_{2,2}^{R} = \boldsymbol{g}_{1,1}^{R}$, such that we obtain the self-consistent relation

$$
\boldsymbol{g}_{1,1}^{R}=\left[E-\boldsymbol{h}-\boldsymbol{v}\cdot\boldsymbol{g}_{1,1}^{R}\cdot\boldsymbol{v}^{\dagger}\right]^{-1}
$$

By solving this equation, we can evalluate the self-energy.

### Problem 1.2.a
For the case of a single-orbital nearest neighbour tight-binding chain rewrite the above equation as a quadratic equation for $g_{1,1}^R$ and solve it analytically. From the two possible solutions, select the retarded one as the solution in which for a complext energy argument, $E + i \eta$ with $\eta \rightarrow 0^+$, the imaginary part of $g_{1,1}^R(E+i\eta)$ is negative.

You should obtain
$$
g^{R}(E)=\frac{1}{t}\left[\left(\frac{E-\epsilon_{0}}{2t}\right)-i\sqrt{1-\left(\frac{E+i0^{+}-\epsilon_{0}}{2t}\right)^{2}}\right]
$$


Study the Green's function for $\left|E-\epsilon_{0}\right|\lessgtr2t$ and make a plot of the real and imaginary parts.

### Problem 1.2.b 
Implement a code that solves the self-consistent equation iterativelly. Compare the numerical result with the analytic one. 
**Hint:** Add a small positive imaginary part to the energy,  $E \rightarrow E + i \eta$, in order to stabelise the iterative method.

## 2. Transport using the Caroli formula

Let us consider a two-contact system described by the Hamiltonian
$$
H_{\text{2-contact}}=\left[\begin{array}{ccc}
\boldsymbol{H}_{C} & \boldsymbol{V}_{C,L} & \boldsymbol{V}_{C,R}\\
\boldsymbol{V}_{L,C} & \boldsymbol{H}_{L}\\
\boldsymbol{V}_{R,C} &  & \boldsymbol{H}_{R}
\end{array}\right]
$$

The Caroli formula expresses the transmitance in terms of Green's functions as A fórmula de Caroli escreve a transmitância como:

$$
\mathcal{T}(E)=\text{Tr}\left[\boldsymbol{G}^{A}(E)\boldsymbol{\Gamma}_{R}(E)\boldsymbol{G}^{R}(E)\boldsymbol{\Gamma}_{L}(E)\right]
$$

where

$$
\boldsymbol{G}^{R}(E)=\left[E-\boldsymbol{H}_{C}-\boldsymbol{\Sigma}_{L}^{R}(E)-\boldsymbol{\Sigma}_{R}^{R}(E)\right]^{-1},
$$

$\boldsymbol{G}^{A}(E)=\left[\boldsymbol{G}^{R}(E)\right]^{\dagger}$ and

$$
\boldsymbol{\Gamma}_{\ell}(E)=i\left[\boldsymbol{\Sigma}_{\ell}^{R}(E)-\boldsymbol{\Sigma}_{\ell}^{A}(E)\right]
$$

### Problem 2.a
Implement the above equations in order to evaluate the transmitance. As an example, evaluate the transmitance through a single-level with energy $\epsilon_0 = 0.5$ that is connected to the left and right contacts with the hoppings $t_{C,L} = t_{C, R} = -0.2$. Compare this with the analytic result.

### Problem 2.b
A sometimes useful approximation is the so called "Wide Band Limit" (WBL) approximation, which consists in ignoring the energy dependence of the leads self energy, and consider its imaginary/anti-hermitian part, i.e,

$$
\Sigma^R_{\ell}(\omega) \simeq -\frac{i}{2}\Gamma_{\ell}(\omega_0)
$$

where $\omega_0$ is some reference energy, choosen to be close to the Fermi levels of the leads. 

Use the WBL for the single-level problem connect to two leads. Make the approximation

$$
\Sigma^R_{\ell}(\omega) \simeq -\frac{i}{2}\Gamma_{\ell}(0)
$$

And compare the result with the previous one.

# 3. Electron-electron interactions at the mean field level

Consider that the Hamiltonian for the centra region is given by

$$
H_C = \sum_{\sigma = \uparrow, \downarrow}\epsilon_0 d_{\sigma}^\dagger d_{\sigma} + U n_{\uparrow} n_{\downarrow}
$$

where $n_\sigma = d_{\sigma}^\dagger d_{\sigma}$ is the electron number operator for spin $\sigma$.

At mean field, we obtain the Hamiltonian

$$
H_C^{MF} = \sum_{\sigma} \epsilon_{\sigma}^{MF} d_{\sigma}^\dagger d_{\sigma}
$$

where $\epsilon_{\sigma}^{MF} = \epsilon_0 + U \left\langle n_{\bar{\sigma}}\right\rangle$

Use the mean field Hamiltonian to evaluate the retarded/advanced Green's functions, the Keldysh equation to express $G^{<}$ and the fact that $\left\langle n_{\sigma}\right\rangle = \int\frac{d\omega}{2\pi} (-i)G^{<}_{\sigma}(\omega)$ to setup a set of self-consistent equations. Solve theses iterativelly. For simplicity, use the WBL approximation.

Use the menad field values obtained from $\left\langle n_{\sigma}\right\rangle$ to evaluate the spin resolved transmittance. Check that for large value of the interation you can have a spin spliting of the transmitance.

As example, consider the following values:

$\epsilon_0 = 0$, $\gamma_L = \gamma_R = 0.005$, $T = 0.025$, $\mu_L = 0$ and $\mu_R = -0.1$